In [ ]:
!wget https://openi.nlm.nih.gov/imgs/collections/NLMCXR_png.tgz

In [ ]:
!mkdir Images

In [ ]:
!tar zxvf /content/NLMCXR_png.tgz -C Images

Streaming output truncated to the last 5000 lines.
./CXR2643_IM-1129-1001.png
./CXR2420_IM-0964-0001-0001.png
./CXR3366_IM-1618-1001.png
./CXR3911_IM-1987-2001.png
./CXR185_IM-0551-2001.png
./CXR1042_IM-0034-3001.png
./CXR2510_IM-1033-1001.png
./CXR779_IM-2321-2001.png
./CXR2117_IM-0745-2001.png
./CXR548_IM-2152-1002.png
./CXR33_IM-1576-13013.png
./CXR1516_IM-0334-2001.png
./CXR3071_IM-1433-1001.png
./CXR3034_IM-1408-2001.png
./CXR429_IM-2070-1001.png
./CXR2172_IM-0786-2001.png
./CXR3589_IM-1767-1001.png
./CXR3594_IM-1772-1002.png
./CXR1632_IM-0413-1001.png
./CXR372_IM-1858-4004.png
./CXR892_IM-2403-2001.png
./CXR2175_IM-0788-1003.png
./CXR1017_IM-0013-1002.png
./CXR222_IM-0823-3003.png
./CXR2931_IM-1334-1001.png
./CXR3486_IM-1695-1001.png
./CXR1102_IM-0069-3001.png
./CXR1315_IM-0204-2001.png
./CXR2969_IM-1360-1001.png
./CXR1297_IM-0195-1001.png
./CXR3772_IM-1891-2001.png
./CXR2042_IM-0686-1001.png
./CXR3216_IM-1520-1001.png
./CXR731_IM-2291-1001.png
./CXR395_IM-2019-1001.png
./CXR3820

In [ ]:
!pip install transformers
!pip install datasets
!pip install rouge_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.3 MB/s eta 0:00:0

In [ ]:
import numpy as np
import torch
import datasets
import os
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms

from transformers import Seq2SeqTrainer ,Seq2SeqTrainingArguments
from transformers import AutoTokenizer, default_data_collator
from transformers import ViTFeatureExtractor, VisionEncoderDecoderModel

In [ ]:
class CONFIG:
  train_batch_size = 4
  test_batch_size = 4
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  epochs = 1
  lr = 1e-5
  max_len = 256
  summary_len = 64
  image_size = (244, 244)
  decoder = 'gpt2'
  encoder = "google/vit-base-patch16-224"


In [ ]:
cfg = CONFIG()

In [ ]:
def special_fun_input(self, token0, token1=None):
  out = [self.bos_token_id] + token0 + [self.eos_token_id]

AutoTokenizer.special_fun_input = special_fun_input

In [ ]:
transform_img = transforms.Compose(
    [
        transforms.Resize(cfg.image_size),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=0,
            std=1
        )
   ]
)

In [ ]:
#rouge function metric

rouge = datasets.load_metric('rouge')

def compute_rouge(pred, target):
  predictions = tokenizer.batch_decode(pred, skip_special_tokens = True)

  target[target==-100] = tokenizer.pad_token_id
  labels = tokenizer.batch_decode(target, skip_special_tokens=True)

  metric = rouge.compute(predictions = predictions, references = labels, rouge_types = ['rouge2'] )["rouge2"].mid

  rouge_result = {'rouge_precision':metric.precision, 'rouge_recall':metric.recall, 'rouge_fmeasure':metric.fmeasure}
  return rouge_result

<ipython-input-7-5076a39bfd77>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = datasets.load_metric('rouge')


In [ ]:
feature_extractor = ViTFeatureExtractor.from_pretrained(cfg.encoder)

tokenizer = AutoTokenizer.from_pretrained(cfg.decoder)
tokenizer.pad_token = tokenizer.unk_token



/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
class Dataset_image(Dataset):
  def __init__(self, caption_df, root, img_feature_extractor, tokenizer, img_transform=None ):
    self.caption_df = caption_df
    self.root_dir = root
    self.tokenizer = tokenizer
    self.feature_extractor = img_feature_extractor
    self.transform = img_transform
    self.max_length = 64

  def __len__(self):
    return len(self.caption_df)

  def __getitem__(self, idx):
    ## I am using a df with two colums; image and caption respectively
    caption = self.caption_df.iloc[idx]['caption']
    image = self.caption_df.iloc[idx]['image']
    image_path = os.path.join(self.root_dir, image)
    # print(idx)
    img = Image.open(image_path).convert('RGB')

    if self.transform is not None:
      img = self.transform(img)
    pixel_values = self.feature_extractor(img, return_tensors='pt').pixel_values
    caption_tokenize = self.tokenizer(caption, padding='max_length',  max_length=self.max_length).input_ids
    caption_tokenize = [caption if caption!=self.tokenizer.pad_token else -100 for caption in caption_tokenize]
    encoding = {'pixel_values': pixel_values.squeeze(), 'labels':torch.tensor(caption_tokenize)}
    return encoding

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
caption = pd.read_csv("/content/drive/MyDrive/archive/captions.csv")
caption.head()

,Unnamed: 0,image,caption,id
0,0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...,0
1,1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .,1
2,2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .,2
3,3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...,3
4,4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...,4


In [ ]:
train_df, val_df = train_test_split(caption, test_size=0.2)

In [ ]:
path = Path("/content/drive/MyDrive/archive/Images")
train_dataset = Dataset_image(train_df, path, feature_extractor, tokenizer, transform_img )
val_dataset = Dataset_image(val_df, path, feature_extractor, tokenizer, transform_img )

In [ ]:
model = VisionEncoderDecoderModel.from_encoder_decoder_pretrained(cfg.encoder, cfg.decoder)

Some weights of the model checkpoint at google/vit-base-patch16-224 were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
# make sure vocab size is set correctly
model.config.vocab_size = model.config.decoder.vocab_size
# set beam search parameters
model.config.eos_token_id = tokenizer.sep_token_id
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.config.max_length = 128
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir='/content',
    per_device_train_batch_size=cfg.train_batch_size,
    per_device_eval_batch_size=cfg.test_batch_size,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=1024,
    save_steps=2048,
    warmup_steps=1024,
    learning_rate = 5e-5,
    #max_steps=1500, # delete for full training
    num_train_epochs = cfg.epochs, #TRAIN_EPOCHS
    overwrite_output_dir=True,
    save_total_limit=1,
)

In [ ]:
# instantiate trainer
trainer = Seq2SeqTrainer(
    tokenizer=feature_extractor,
    model=model,
    args=training_args,
    compute_metrics=compute_rouge,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=default_data_collator,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


6424
7429
16986
9734


Epoch,Training Loss,Validation Loss


20364
2122
23838
7364
10559
23697
4510
12028
19969
9032
7455
15611
13383
10747
13094
10168
5841
21000
11611
7082


KeyboardInterrupt: ignored